In [1]:
import pandas as pd
import numpy as np

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

# Basic import
import eugene as eu
eu.__version__

Global seed set to 13


'0.0.6'

In [4]:
from modiscolite.tfmodisco import TFMoDISco

In [16]:
sdata = eu.datasets.random1000()
eu.pp.ohe_seqs_sdata(sdata)
model = eu.models.DeepBind(input_len=100, output_dim=1)
eu.interpret.feature_attribution_sdata(model, sdata, method='DeepLift')

One-hot encoding sequences:   0%|          | 0/1000 [00:00<?, ?it/s]

SeqData object modified:
	ohe_seqs: None -> 1000 ohe_seqs added
No transforms given, assuming just need to tensorize.


Computing saliency on batches of size 128:   0%|          | 0/7 [00:00<?, ?it/s]

/home/vscode/.local/lib/python3.8/site-packages/captum/attr/_core/deep_lift.py:336: UserWarning: Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished
  warnings.warn(
/home/vscode/.local/lib/python3.8/site-packages/captum/attr/_core/deep_lift.py:467: UserWarning: An invalid module MaxPool1d(kernel_size=85, stride=85, padding=0, dilation=1, ceil_mode=False) is detected. Saved gradients will
                be used as the gradients of the module's input tensor.
                See MaxPool1d as an example.
  warnings.warn(
/home/vscode/.local/lib/python3.8/site-packages/torch/nn/functional.py:651: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool1d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


In [17]:
sdata

SeqData object with = 1000 seqs
seqs = (1000,)
names = (1000,)
rev_seqs = None
ohe_seqs = (1000, 4, 100)
ohe_rev_seqs = None
seqs_annot: 'label_0', 'label_1', 'label_2', 'label_3', 'label_4', 'label_5', 'label_6', 'label_7', 'label_8', 'label_9', 'activity_0', 'activity_1', 'activity_2', 'activity_3', 'activity_4', 'activity_5', 'activity_6', 'activity_7', 'activity_8', 'activity_9'
pos_annot: None
seqsm: None
uns: 'DeepLift_imps'

In [18]:
seqs = sdata.ohe_seqs
imps = sdata.uns["DeepLift_imps"]
seqs.shape, imps.shape

((1000, 4, 100), (1000, 4, 100))

In [23]:
pos_patterns, neg_patterns = TFMoDISco(
    one_hot=seqs.transpose(0, 2, 1),
    hypothetical_contribs=imps.transpose(0, 2, 1)
)

In [28]:
neg_patterns

In [29]:
sset = pos_patterns[0]

In [35]:
seqlet = sset.seqlets[0]

In [41]:
seqlet.start, seqlet.end, seqlet.example_idx, seqlet.is_revcomp, seqlet.sequence

(55,
 85,
 126,
 True,
 array([[0, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 0]], dtype=int8))

In [42]:
print(seqlet)

example:126,start:55,end:85,rc:True


In [48]:
np.save('seqs.npy', seqs)
np.save('imps.npy', imps)

In [24]:
TFMoDISco?

Signature:
TFMoDISco(
    one_hot,
    hypothetical_contribs,
    sliding_window_size=21,
    flank_size=10,
    min_metacluster_size=100,
    weak_threshold_for_counting_sign=0.8,
    max_seqlets_per_metacluster=20000,
    target_seqlet_fdr=0.2,
    min_passing_windows_frac=0.03,
    max_passing_windows_frac=0.2,
    n_leiden_runs=50,
    n_leiden_iterations=-1,
    min_overlap_while_sliding=0.7,
    nearest_neighbors_to_compute=500,
    affmat_correlation_threshold=0.15,
    tsne_perplexity=10.0,
    frac_support_to_trim_to=0.2,
    min_num_to_trim_to=30,
    trim_to_window_size=20,
    initial_flank_to_add=5,
    prob_and_pertrack_sim_merge_thresholds=[(0.8, 0.8), (0.5, 0.85), (0.2, 0.9)],
    prob_and_pertrack_sim_dealbreaker_thresholds=[(0.4, 0.75), (0.2, 0.8), (0.1, 0.85), (0.0, 0.9)],
    subcluster_perplexity=50,
    merging_max_seqlets_subsample=300,
    final_min_cluster_size=20,
    min_ic_in_window=0.6,
    min_ic_windowsize=6,
    ppm_pseudocount=0.001,
    verbose=False,


In [3]:
import h5py

In [4]:
results = h5py.File('modisco_results.h5', 'w')

In [8]:
results.keys()

<KeysViewHDF5 []>

In [6]:
results["neg_patterns"]

KeyError: "Unable to open object (object 'neg_patterns' doesn't exist)"